In [1]:
from config import Config
from dataset import create_dataset
from model import Model
import tensorflow as tf

config = Config("config.yaml")
config

2023-03-08 16:54:50.886975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 16:54:50.987182: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-08 16:54:50.990538: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 16:54:50.990550: I tensorflow/compiler/xla/stream_executor/cuda/cudar

Config: {'epochs': 2, 'compression': 'GZIP', 'global_batch_size': 8192, 'label': 'label', 'shuffle_buffer': 10, 'train_path': 'gs://mgaiduk-us-central1/ratings/csv_gzip/part*', 'validate_path': 'gs://mgaiduk-us-central1/ratings_validate/csv_gzip/part*', 'save_model_path': 'gs://mgaiduk-us-central1/models/model1', 'train_rows': 65536, 'trainval_rows': 8192, 'eval_rows': 65536, 'model': Model: {'learning_rate': 0.01, 'features': [Feature: {'hash': True, 'vocab_size': 25000000, 'embedding_dim': 16, 'name': 'userId', 'belongs_to': 'user'}, Feature: {'hash': True, 'vocab_size': 5000000, 'embedding_dim': 16, 'name': 'movieId', 'belongs_to': 'movie'}]}, 'dataset_features': [DatasetFeature: {'name': 'userId', 'type': 'string'}, DatasetFeature: {'name': 'movieId', 'type': 'string'}, DatasetFeature: {'name': 'label', 'type': 'int'}, DatasetFeature: {'name': 'timestamp', 'type': 'int'}], 'cycle_length': 8}

In [2]:
strategy = tf.distribute.get_strategy()
train_dataset = create_dataset(config, strategy, config.train_path)

2023-03-08 16:54:52.122642: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-08 16:54:52.122666: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-08 16:54:52.122680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mgaiduk-cpu-vm): /proc/driver/nvidia/version does not exist
2023-03-08 16:54:52.122903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
for elem in train_dataset:
    break
elem

({'userId': <tf.Tensor: shape=(8192,), dtype=int64, numpy=array([10348088, 15265920,  3085306, ...,  7197379,  5566271, 12232036])>,
  'movieId': <tf.Tensor: shape=(8192,), dtype=int64, numpy=array([2963232, 2339632, 1657084, ..., 1070069,  825906,  726723])>},
 {'label': <tf.Tensor: shape=(8192,), dtype=int64, numpy=array([2, 2, 2, ..., 2, 2, 2])>})

In [4]:
with strategy.scope():
    model = Model(config)
    model.compile(model.optimizer, steps_per_execution=1)
model(elem[0])

{'label': <tf.Tensor: shape=(8192, 1), dtype=float32, numpy=
 array([[0.5796528 ],
        [0.3376486 ],
        [0.41295588],
        ...,
        [0.5490542 ],
        [0.35019007],
        [0.48135453]], dtype=float32)>}

In [5]:
model.compute_loss(elem)

<tf.Tensor: shape=(), dtype=float32, numpy=0.71205956>

In [ ]:
with strategy.scope():
    model.fit(train_dataset, steps_per_epoch=10)

 5/10 [==============>...............] - ETA: 6s - label-crossentropy: 0.6848 - auc: 0.0000e+00 - pr-auc: 1.0000 - accuracy: 0.0000e+00 - prediction_mean: 0.5040 - label_mean: 2.0000 - loss: 0.6848 - regularization_loss: 0.0000e+00 - total_loss: 0.6848

In [ ]:
model.compute_loss(elem)